# 05tools_4: Experiment Comparisons

Use the Python client to retrieve, review, and compare experiments from this series

### Prerequisites:
-  At least 1 of the notebooks in this series [05, 05a-05i]
    - Each of these notebooks creates an experiment.  Rerunning the notebooks creates additonal runs of the experiment.  The models created by each run are uploaded to the Vertex AI Model Registry with a version alias for the experiment run name and has labels for the experiment_name and run_name.

---
## Setup

inputs:

In [143]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [144]:
REGION = 'us-central1'
SERIES = '05'

packages:

In [145]:
from google.cloud import aiplatform
import pandas as pd
from IPython.display import HTML

clients:

In [146]:
aiplatform.init(project = PROJECT_ID, location = REGION)

---
## Experiments

Get a list of all experiments in this project:

In [147]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [148]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

### Combine Runs from All Experiments in SERIES

Combine the runs from all experiments in SERIES into a single dataframe:

In [149]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05c-classification-dnn
experiment-05-05b-classification-dnn
experiment-05-05a-classification-dnn
experiment-05-05-classification-dnn


In [150]:
results

,experiment_name,run_name,run_type,state,param.MODEL_VERSIONED_RESOURCE_NAME,param.job.resource_name,param.SHUFFLE,param.nclasses,param.DATANAME,param.NOTEBOOK,...,metric.train_accuracy,metric.val_accuracy,metric.train_loss,metric.train_auprc,time_series_metric.train_accuracy,time_series_metric.train_loss,time_series_metric.val_accuracy,time_series_metric.train_auprc,time_series_metric.val_auprc,time_series_metric.val_loss
0,experiment-05-05c-classification-dnn,run-20220820145839,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,projects/1026793852137/locations/us-central1/c...,1000.0,2.0,fraud,05c,...,0.999382,0.999256,0.003817,0.999580,0.999373,0.003501,0.999256,0.999686,0.999762,0.004752
0,experiment-05-05b-classification-dnn,run-20220820135734,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,projects/1026793852137/locations/us-central1/c...,1000.0,2.0,fraud,05b,...,0.999426,0.999398,0.003640,0.999576,0.999399,0.003174,0.999398,0.999692,0.999530,0.005067
0,experiment-05-05a-classification-dnn,run-20220819202407,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,projects/1026793852137/locations/us-central1/c...,1000.0,2.0,fraud,05a,...,0.999399,0.999256,0.003784,0.999587,0.999360,0.003388,0.999256,0.999703,0.999624,0.005430
1,experiment-05-05a-classification-dnn,run-20220819200415,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,projects/1026793852137/locations/us-central1/c...,1000.0,2.0,fraud,05a,...,0.999434,0.999292,0.003726,0.999592,0.999430,0.003390,0.999292,0.999692,0.999577,0.005310
0,experiment-05-05-classification-dnn,run-20220820134653,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,NaN,1000.0,2.0,fraud,05,...,0.999360,0.999221,0.005110,0.999446,0.999329,0.004197,0.999221,0.999619,0.999388,0.006359
1,experiment-05-05-classification-dnn,run-20220819201639,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,NaN,1000.0,2.0,fraud,05,...,0.999233,0.999115,0.005703,0.999418,0.999136,0.004978,0.999115,0.999557,0.999384,0.006003
2,experiment-05-05-classification-dnn,run-20220819200009,system.ExperimentRun,COMPLETE,projects/1026793852137/locations/us-central1/m...,NaN,1000.0,2.0,fraud,05,...,0.999382,0.999221,0.005540,0.999372,0.999277,0.004442,0.999221,0.999562,0.999342,0.006728


Get Links to all Experiment Runs with Vertex AI Training Jobs:
- Link to Job
- Link to TensorBoard for Job

In [151]:
# just names and job* columns
mask = results.columns.str.startswith('param.job')
mask[0:2] = True
HTML(results.loc[:,mask].to_html(render_links=True, escape=False))

,experiment_name,run_name,param.job.resource_name,param.job.board_link,param.job.display_name,param.job.job_link,param.job.type
0,experiment-05-05c-classification-dnn,run-20220820145839,projects/1026793852137/locations/us-central1/customJobs/678650737378459648,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+678650737378459648,05c_fraud_20220820145839,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/678650737378459648/cpu?cloudshell=false&project=statmike-mlops-349915,aiplatform.CustomJob.from_local_script()
0,experiment-05-05b-classification-dnn,run-20220820135734,projects/1026793852137/locations/us-central1/customJobs/5835835260671098880,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+5835835260671098880,05b_fraud_20220820135734,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/5835835260671098880/cpu?cloudshell=false&project=statmike-mlops-349915,aiplatform.CustomJob.from_local_script()
0,experiment-05-05a-classification-dnn,run-20220819202407,projects/1026793852137/locations/us-central1/customJobs/9194368294503579648,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+9194368294503579648,05a_fraud_20220819202407,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/9194368294503579648/cpu?cloudshell=false&project=statmike-mlops-349915,aiplatform.CustomJob.from_local_script()
1,experiment-05-05a-classification-dnn,run-20220819200415,projects/1026793852137/locations/us-central1/customJobs/8012173392318824448,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+8012173392318824448,05a_fraud_20220819200415,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/8012173392318824448/cpu?cloudshell=false&project=statmike-mlops-349915,aiplatform.CustomJob.from_local_script()
0,experiment-05-05-classification-dnn,run-20220820134653,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05-classification-dnn,run-20220819201639,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05-classification-dnn,run-20220819200009,NaN,NaN,NaN,NaN,NaN


### Hyperlinks To TensorBoard for Experiments and Comparison Of All Experiments

Get a hyperlink to the TensorBoard for each experiment:

In [152]:
tensorboards = []
for experiment in experiments:
    tboard = experiment.get_backing_tensorboard_resource().resource_name.replace('/', '+')
    tensorboards.append([experiment.name, f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tboard}+experiments+{experiment.name}"])
tensorboards = pd.DataFrame(tensorboards, columns = ['Experiment', 'TensorBoard Link'])    

In [153]:
HTML(tensorboards.to_html(render_links=True, escape=False))

,Experiment,TensorBoard Link
0,experiment-05-05c-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+experiment-05-05c-classification-dnn
1,experiment-05-05b-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+experiment-05-05b-classification-dnn
2,experiment-05-05a-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+experiment-05-05a-classification-dnn
3,experiment-05-05-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+4204796347406090240+experiments+experiment-05-05-classification-dnn


Compare all experiments in SERIES with TensorBoard:

In [154]:
compare_link = f"https://{REGION}.tensorboard.googleusercontent.com/compare/"
for e, experiment in enumerate(experiments):
    if e>0: compare_link += ','
    tboard = experiment.get_backing_tensorboard_resource().resource_name.split('/')
    compare_link += f"{e+1}-{experiment.name}:{'+'.join(tboard[1::2])}+{experiment.name}"
print(compare_link)

https://us-central1.tensorboard.googleusercontent.com/compare/1-experiment-05-05c-classification-dnn:1026793852137+us-central1+4204796347406090240+experiment-05-05c-classification-dnn,2-experiment-05-05b-classification-dnn:1026793852137+us-central1+4204796347406090240+experiment-05-05b-classification-dnn,3-experiment-05-05a-classification-dnn:1026793852137+us-central1+4204796347406090240+experiment-05-05a-classification-dnn,4-experiment-05-05-classification-dnn:1026793852137+us-central1+4204796347406090240+experiment-05-05-classification-dnn
